In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import numpy as np
from datetime import datetime

In [ ]:
# Root folder. Change this according to your folder structure.
ROOT_DIR = '/deeplearning/'

In [ ]:
zipped_data = os.path.join(ROOT_DIR, 'cse7512-00-dl-tp-pt1.zip')

data_dir = '/content/data'
os.makedirs(data_dir, exist_ok=True)
!cp '{zipped_data}' '{data_dir}'

In [ ]:
!unzip -qq '{data_dir}/cse7512-00-dl-tp-pt1.zip' -d '{data_dir}'

In [ ]:
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Model
 Define your model here.

## ResNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

        # self.activation = nn.ReLU()
        self.activation = nn.ELU()

    def forward(self, x):
        out = self.activation(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.activation(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 96, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 126, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 160, num_blocks[3], stride=2)

        self.linear1 = nn.Linear(1440*block.expansion, 64)
        self.classifier = nn.Linear(64, num_classes)

        # self.activation = nn.ReLU()
        self.activation = nn.ELU()
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):

        out = self.activation(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)

        # out = out.view(out.size(0), -1)
        out = out.reshape(out.size(0), out.size(1)*out.size(2)*out.size(3))
        
        out = self.activation(self.linear1(out))
        out = self.classifier(out)
        
        return out


# Utils

In [ ]:
# Get your model here. This function returns the actual model and the name. The name is not important and can be an empty string.
def Model():
    return ResNet(BasicBlock, [2, 2, 2, 2]), 'ResNet'

In [ ]:
# Some notes for logging purposes.
note = 'ResNet_v6. Random affine, perspective, and erasing are included. Random erasing area increased abit.'

# Check

In [ ]:
# Shape check. Check if the model is outputing proper shapes.
model, model_architecture = Model()
y = model(torch.randn(1, 3, 96, 96))
print(y.size())

In [ ]:
# Check number of parameters of your model.
model, model_architecture = Model()
pytorch_total_params = sum(p.numel() for p in model.parameters())

print(f"Number of parameters: {pytorch_total_params}")

print(int(pytorch_total_params))
print(2000000)

assert int(pytorch_total_params) <= 2000000, 'Your model has the number of parameters more than 2 millions..'

In [ ]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            # correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
# Util parameters.
# current_time = '20201115035927'                               # Use this one if you want to continue from last checkpoint.
current_time = datetime.now().strftime('%Y%m%d%H%M%S')          # Use this one when for training from scratch.
SAVEPATH = os.path.join(ROOT_DIR, 'checkpoints', str(current_time) + '_full_data') # Save path for model, logs and note.
LOG_DIR = os.path.join(SAVEPATH, 'logs')
os.makedirs(SAVEPATH, exist_ok=True)

PRINTFREQ = 10

# Train Model

In [ ]:
import time

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
def train(train_loader, epoch, model, optimizer, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, data_time, losses,
                             top1, top5, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input = input.cuda()
        target = target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss, accuracy 
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINTFREQ == 0:
            progress.print(i)

    print('Training => Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    return top1.avg, top5.avg, losses.avg

In [ ]:
# Load tensorboard. 
%load_ext tensorboard
%tensorboard --logdir '{LOG_DIR}'

# After a few minutes into the training, reload the below tensorboard to see your progress.

# Hyperparameters


In [ ]:
BATCHSIZE = 64
INITIAL_EPOCH = 0
# INITIAL_EPOCH = 2299 + 1  # For training from a checkpoint.
EPOCHS = 1840
LR = 1e-4

# Training

In [ ]:
model, model_architecture = Model()
model = model.cuda()

# model_name = 'train_acc98.7000_train_loss_0.0360_epoch_2299.pth'              # Use for training from a checkpoint.
# model.load_state_dict(torch.load(os.path.join(SAVEPATH, model_name)))         # Load checkpoint model.

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.cuda()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Training augmentations.
train_transform = transforms.Compose([
    transforms.RandomCrop(96, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=(0.5, 2.5), contrast=(0.5, 2.5), saturation=(0.5, 2.5), hue=(-0.1, 0.1)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.2, 0.2), ratio=(1, 1)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), shear=5),
    transforms.RandomPerspective(distortion_scale=0.25),
    normalize
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset,
                            batch_size=BATCHSIZE, shuffle=True,
                            num_workers=0, pin_memory=True)

# Parameters for logging.
writer = SummaryWriter(LOG_DIR)

top1_acc_train = None
top5_acc_train = None
loss_train = None

In [ ]:
for epoch in range(INITIAL_EPOCH, EPOCHS):

    print("\n----- epoch: {}, lr: {} -----".format(
        epoch, optimizer.param_groups[0]["lr"]))
    
    start_time = time.time()

    # Train
    top1_acc_train, top5_acc_train, loss_train = train(train_loader, epoch, model, optimizer, criterion)
    writer.add_scalar('accuracy/top1_train', top1_acc_train, epoch)
    writer.add_scalar('accuracy/top5_train', top5_acc_train, epoch)
    writer.add_scalar('loss/train', loss_train, epoch)

    elapsed_time = time.time() - start_time
    print('==> {:.2f} seconds to train this epoch\n'.format(elapsed_time))

    model_name = 'train_acc{:.4f}_train_loss_{:.4f}_epoch_{}.pth'.format(top1_acc_train, loss_train, epoch)
    torch.save(model.state_dict(), os.path.join(SAVEPATH, model_name))

    # learning rate scheduling
    # scheduler.step()


In [ ]:
print(f"Train Top-1 Accuracy: {top1_acc_train}")

In [ ]:
def save_model_info():
    info_file = os.path.join(SAVEPATH, 'info.txt')
    info = '''
    MODEL : {}
    EPOCHS : {}\n 
    optimizer : {}\n
    criterion : {}\n
    train_transform : {}\n\n

    train_acc_top1 : {}\n
    loss_train : {}\n

    pytorch_total_params : {}\n\n
    
    note : {}\n
    '''.format(model_architecture, EPOCHS, optimizer, criterion, train_transform, top1_acc_train, loss_train, pytorch_total_params, note)
    
    with open(info_file, 'w') as f:
        f.write(info)

In [ ]:
save_model_info()

# Make an evalutation csv file

In [ ]:
epochs = EPOCHS - 1 # Just for using in the submission file name.

# Use this to output submission file from your desire model checkpoint.

# model_name = 'train_acc98.4600_train_loss_0.0450_epoch_2355.pth'                      
# SAVEPATH = os.path.join(ROOT_DIR, 'checkpoints', '20201106080213' + '_full_data')
# epochs = 2355

In [ ]:
import torch
import pandas as pd
import argparse
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
def eval():
    ########## You can change this part only in this cell ##########
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])
    ################################################################

    test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCHSIZE, num_workers=0, shuffle=False)

    model, _ = Model()
    model = model.cuda()

    # I just changed the model path here.
    model.load_state_dict(torch.load(os.path.join(SAVEPATH, model_name)))

    print('Make an evaluation csv file for kaggle submission...')
    Category = []
    for input, _ in test_loader:
        input = input.cuda()
        output = model(input)
        output = torch.argmax(output, dim=1)
        Category = Category + output.tolist()

    Id = list(range(0, 8000))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    # I just changed the submission file name here.
    df.to_csv(os.path.join(SAVEPATH, 'submission_epochs_{}.csv'.format(epochs)), index=False)
    print('Done!!')


if __name__ == "__main__":
    eval()